# 8. Работа с базами данных в Python.

## 8.1. TinyDB

https://pypi.python.org/pypi/tinydb

Установка:

    pip install tinydb

In [1]:
import tinydb
db = tinydb.TinyDB('tinydb.json')
db.insert({'int': 1, 'char': 'a'})
db.insert({'int': 1, 'char': 'b'})

print(db.all())

[{'int': 1, 'char': 'a'}, {'int': 1, 'char': 'b'}, {'int': 1, 'char': 'a'}, {'int': 1, 'char': 'b'}, {'int': 1, 'char': 'a'}, {'int': 1, 'char': 'b'}]


In [2]:
%%bash
cat tinydb.json

{"_default": {"1": {"int": 1, "char": "a"}, "2": {"int": 1, "char": "b"}, "3": {"int": 1, "char": "a"}, "4": {"int": 1, "char": "b"}, "5": {"int": 1, "char": "a"}, "6": {"int": 1, "char": "b"}}, "metrics": {"1": {"signal": "system2", "timestamp": 1492707412, "value": -52, "comment": "Update"}, "2": {"signal": "system1", "timestamp": 1492707502, "value": 324, "comment": "Commit"}, "3": {"signal": "system2", "timestamp": 1492707602, "value": -52, "comment": "Release"}, "4": {"signal": "system2", "timestamp": 1492707702, "value": 512, "comment": "Fix"}, "5": {"signal": "system2", "timestamp": 1492707412, "value": -52, "comment": "Update"}, "6": {"signal": "system1", "timestamp": 1492707502, "value": 324, "comment": "Commit"}, "7": {"signal": "system2", "timestamp": 1492707602, "value": -52, "comment": "Release"}, "8": {"signal": "system2", "timestamp": 1492707702, "value": 512, "comment": "Fix"}, "9": {"signal": "system2", "timestamp": 1492707412, "value": -52, "comment": "Update"}, "10":

In [3]:
import tinydb
db = tinydb.TinyDB('tinydb.json')

table = db.table('metrics')
table.insert({'signal': 'system2', 'timestamp': 1492707412, 'value': -52, 'comment': 'Update'})
table.insert({'signal': 'system1', 'timestamp': 1492707502, 'value': 324, 'comment': 'Commit'})
table.insert({'signal': 'system2', 'timestamp': 1492707602, 'value': -52, 'comment': 'Release'})
table.insert({'signal': 'system2', 'timestamp': 1492707702, 'value': 512, 'comment': 'Fix'})

print(db.all(), table.all())

[{'int': 1, 'char': 'a'}, {'int': 1, 'char': 'b'}, {'int': 1, 'char': 'a'}, {'int': 1, 'char': 'b'}, {'int': 1, 'char': 'a'}, {'int': 1, 'char': 'b'}] [{'signal': 'system2', 'timestamp': 1492707412, 'value': -52, 'comment': 'Update'}, {'signal': 'system1', 'timestamp': 1492707502, 'value': 324, 'comment': 'Commit'}, {'signal': 'system2', 'timestamp': 1492707602, 'value': -52, 'comment': 'Release'}, {'signal': 'system2', 'timestamp': 1492707702, 'value': 512, 'comment': 'Fix'}, {'signal': 'system2', 'timestamp': 1492707412, 'value': -52, 'comment': 'Update'}, {'signal': 'system1', 'timestamp': 1492707502, 'value': 324, 'comment': 'Commit'}, {'signal': 'system2', 'timestamp': 1492707602, 'value': -52, 'comment': 'Release'}, {'signal': 'system2', 'timestamp': 1492707702, 'value': 512, 'comment': 'Fix'}, {'signal': 'system2', 'timestamp': 1492707412, 'value': -52, 'comment': 'Update'}, {'signal': 'system1', 'timestamp': 1492707502, 'value': 324, 'comment': 'Commit'}, {'signal': 'system2', 

In [4]:
%%bash
cat tinydb.json

{"_default": {"1": {"int": 1, "char": "a"}, "2": {"int": 1, "char": "b"}, "3": {"int": 1, "char": "a"}, "4": {"int": 1, "char": "b"}, "5": {"int": 1, "char": "a"}, "6": {"int": 1, "char": "b"}}, "metrics": {"1": {"signal": "system2", "timestamp": 1492707412, "value": -52, "comment": "Update"}, "2": {"signal": "system1", "timestamp": 1492707502, "value": 324, "comment": "Commit"}, "3": {"signal": "system2", "timestamp": 1492707602, "value": -52, "comment": "Release"}, "4": {"signal": "system2", "timestamp": 1492707702, "value": 512, "comment": "Fix"}, "5": {"signal": "system2", "timestamp": 1492707412, "value": -52, "comment": "Update"}, "6": {"signal": "system1", "timestamp": 1492707502, "value": 324, "comment": "Commit"}, "7": {"signal": "system2", "timestamp": 1492707602, "value": -52, "comment": "Release"}, "8": {"signal": "system2", "timestamp": 1492707702, "value": 512, "comment": "Fix"}, "9": {"signal": "system2", "timestamp": 1492707412, "value": -52, "comment": "Update"}, "10":

In [5]:
import tinydb
db = tinydb.TinyDB('tinydb.json')

table = db.table('metrics')
rec = tinydb.Query()

res1 = table.search(rec.signal == 'system2')
print(res1)

# Скорки в условии обязательны
res2 = table.search((rec.signal == 'system2') & (rec.comment == 'Update'))
print(res2)

[{'signal': 'system2', 'timestamp': 1492707412, 'value': -52, 'comment': 'Update'}, {'signal': 'system2', 'timestamp': 1492707602, 'value': -52, 'comment': 'Release'}, {'signal': 'system2', 'timestamp': 1492707702, 'value': 512, 'comment': 'Fix'}, {'signal': 'system2', 'timestamp': 1492707412, 'value': -52, 'comment': 'Update'}, {'signal': 'system2', 'timestamp': 1492707602, 'value': -52, 'comment': 'Release'}, {'signal': 'system2', 'timestamp': 1492707702, 'value': 512, 'comment': 'Fix'}, {'signal': 'system2', 'timestamp': 1492707412, 'value': -52, 'comment': 'Update'}, {'signal': 'system2', 'timestamp': 1492707602, 'value': -52, 'comment': 'Release'}, {'signal': 'system2', 'timestamp': 1492707702, 'value': 512, 'comment': 'Fix'}, {'signal': 'system2', 'timestamp': 1492707412, 'value': -52, 'comment': 'Update'}, {'signal': 'system2', 'timestamp': 1492707602, 'value': -52, 'comment': 'Release'}, {'signal': 'system2', 'timestamp': 1492707702, 'value': 512, 'comment': 'Fix'}]
[{'signal':

## 8.2. sqlite

https://docs.python.org/3/library/sqlite3.html

In [6]:
import sqlite3

conn = sqlite3.connect('example.db')
cur = conn.cursor()

cur.execute('''CREATE TABLE metrics
             (signal text, timestamp real, value real, comment text)''')

conn.close()

In [7]:
conn = sqlite3.connect('example.db')
cur = conn.cursor()

cur.execute("INSERT INTO metrics VALUES ('system1', 1492707402, 12, 'New event')")

conn.commit()
conn.close()

In [8]:
%%bash
xxd example.db | head -n 2
echo "                         ***"
xxd example.db | tail -n 5

0000000: 5351 4c69 7465 2066 6f72 6d61 7420 3300  SQLite format 3.
0000010: 1000 0101 0040 2020 0000 0002 0000 0002  .....@  ........
                         ***
0001fb0: 0000 0000 0000 0000 0000 0000 0000 0000  ................
0001fc0: 0000 0000 0000 0000 0000 0000 0000 0000  ................
0001fd0: 0000 0000 0000 0000 0000 0000 0000 0000  ................
0001fe0: 0000 0000 1a01 051b 0401 1f73 7973 7465  ...........syste
0001ff0: 6d31 58f8 e84a 0c4e 6577 2065 7665 6e74  m1X..J.New event


In [9]:
conn = sqlite3.connect('example.db')
cur = conn.cursor()

values = [
    ('system2', 1492707412, -52, 'Update'),
    ('system1', 1492707502, 324, 'Commit'),
    ('system2', 1492707602, -52, 'Release'),
    ('system2', 1492707702, 512, 'Fix')
]

cur.executemany('INSERT INTO metrics VALUES (?,?,?,?)', values)

conn.commit()
conn.close()

In [10]:
conn = sqlite3.connect('example.db')
cur = conn.cursor()

label = ('system2',)
cur.execute('SELECT * FROM metrics WHERE signal LIKE ?', label)

print(cur.fetchone())
print(cur.fetchall())

conn.close()

('system2', 1492707412.0, -52.0, 'Update')
[('system2', 1492707602.0, -52.0, 'Release'), ('system2', 1492707702.0, 512.0, 'Fix')]


In [11]:
conn = sqlite3.connect('example.db')
cur = conn.cursor()

label = ('system2',)

for res in cur.execute('SELECT * FROM metrics WHERE signal LIKE ?', label):
    print(res)

conn.close()

('system2', 1492707412.0, -52.0, 'Update')
('system2', 1492707602.0, -52.0, 'Release')
('system2', 1492707702.0, 512.0, 'Fix')


## 8.3. PostgreSQL


    # docker run -e POSTGRES_DB=pydb -e POSTGRES_USER=pydb -e POSTGRES_PASSWORD=pydb -d --name pydb -p 5432:5432 postgres
    
Установка модуля для работы с базой данных:

    pip install psycopg2

или

    conda install psycopg2
    
Документация: http://initd.org/psycopg/docs/

In [12]:
import psycopg2

conn = psycopg2.connect(
    dbname='pydb',
    user='pydb',
    password='pydb',
    host='localhost',
    port=5432
)

cur = conn.cursor()

cur.execute("CREATE TABLE metrics2 (signal varchar, timestamp integer, value integer, comment varchar);")
conn.commit()

cur.close()
conn.close() 

In [13]:
import psycopg2

conn = psycopg2.connect(
    dbname='pydb',
    user='pydb',
    password='pydb',
    host='localhost',
    port=5432
)

cur = conn.cursor()

values = [
    ('system2', 1492707412, -52, 'Update'),
    ('system1', 1492707502, 324, 'Commit'),
    ('system2', 1492707602, -52, 'Release'),
    ('system2', 1492707702, 512, 'Fix')
]

cur.executemany('INSERT INTO metrics2 VALUES (%s, %s, %s, %s)', values)

conn.commit()

cur.close()
conn.close() 

In [14]:
import psycopg2

conn = psycopg2.connect(
    dbname='pydb',
    user='pydb',
    password='pydb',
    host='localhost',
    port=5432
)

cur = conn.cursor()

cur.execute("SELECT * FROM metrics2 WHERE signal LIKE %s;", ('system2', ))
print(cur.fetchone())
print(cur.fetchall())


cur.close()
conn.close()

('system2', 1492707412, -52, 'Update')
[('system2', 1492707602, -52, 'Release'), ('system2', 1492707702, 512, 'Fix')]


## 8.4. MongoDB

Запуск MongoDB в Docker:

    docker run --name pymongo -d -p 27017:27017 mongo

Установка модуля для работы с MongoDB:

    pip install pymongo
    conda install pymongo

Документация: http://api.mongodb.com/python/3.4.0/tutorial.html

In [15]:
import pymongo

client = pymongo.MongoClient('localhost', 27017)
db = client['metrics-collection']

entry = {
    'signal': 'system2',
    'timestamp': 1492707412,
    'value': -52,
    'comment': 'Update'
}

metrics = db.metrics
entry_id = metrics.insert_one(entry).inserted_id
print(entry_id)

58f9ca7b326299550eb89665


In [16]:
db.collection_names(include_system_collections=False)

['metrics']

In [17]:
metrics.find_one({'signal': 'system2'})

{'_id': ObjectId('58f9ca7b326299550eb89665'),
 'comment': 'Update',
 'signal': 'system2',
 'timestamp': 1492707412,
 'value': -52}

In [18]:
import pymongo

client = pymongo.MongoClient('localhost', 27017)
db = client['metrics-collection']

entries = [
    {'signal': 'system2', 'timestamp': 1492707412, 'value': -52, 'comment': 'Update'},
    {'signal': 'system1', 'timestamp': 1492707502, 'value': 324, 'comment': 'Commit'},
    {'signal': 'system2', 'timestamp': 1492707602, 'value': -52, 'comment': 'Release'},
    {'signal': 'system2', 'timestamp': 1492707702, 'value': 512, 'comment': 'Fix'}
]

metrics = db.metrics
metrics.insert_many(entries)

In [19]:
for signal in metrics.find({'signal': 'system2'}):
    print(signal)

{'_id': ObjectId('58f9ca7b326299550eb89665'), 'signal': 'system2', 'timestamp': 1492707412, 'value': -52, 'comment': 'Update'}
{'_id': ObjectId('58f9ca7c326299550eb89667'), 'signal': 'system2', 'timestamp': 1492707412, 'value': -52, 'comment': 'Update'}
{'_id': ObjectId('58f9ca7c326299550eb89669'), 'signal': 'system2', 'timestamp': 1492707602, 'value': -52, 'comment': 'Release'}
{'_id': ObjectId('58f9ca7c326299550eb8966a'), 'signal': 'system2', 'timestamp': 1492707702, 'value': 512, 'comment': 'Fix'}


## 8.5. SQLAlchemy

Установка:

    pip install sqlalchemy
    conda install sqlalchemy
    
Документация: http://docs.sqlalchemy.org/en/rel_1_1

Рассмотрим урок http://docs.sqlalchemy.org/en/rel_1_1/orm/tutorial.html

In [20]:
import sqlalchemy
engine = sqlalchemy.create_engine('sqlite:///alchemy.db', echo=True)

В классе должны быть заданы атрибут `__tablename__` и как минимум один `Column` с опцией `primary_key`.

In [21]:
from sqlalchemy.ext import declarative
Base = declarative.declarative_base()

class User(Base):
    __tablename__ = 'users'

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    name = sqlalchemy.Column(sqlalchemy.String)
    fullname = sqlalchemy.Column(sqlalchemy.String)
    password = sqlalchemy.Column(sqlalchemy.String)

    def __repr__(self):
        tmpl = "<User(name='{}', fullname='{}', password='{}')>"
        return  tmpl.format(self.name, self.fullname, self.password)

Base.metadata.create_all(engine)

2017-04-21 12:01:48,347 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-04-21 12:01:48,348 INFO sqlalchemy.engine.base.Engine ()
2017-04-21 12:01:48,350 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-04-21 12:01:48,351 INFO sqlalchemy.engine.base.Engine ()
2017-04-21 12:01:48,353 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-04-21 12:01:48,354 INFO sqlalchemy.engine.base.Engine ()
2017-04-21 12:01:48,357 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2017-04-21 12:01:48,357 INFO sqlalchemy.engine.base.Engine ()
2017-04-21 12:01:48,386 INFO sqlalchemy.engine.base.Engine COMMIT


In [22]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')

In [23]:
from sqlalchemy import orm

SessionMaker = orm.sessionmaker(bind=engine)
session = SessionMaker()
session.add(ed_user)

In [24]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', password='foobar'),
    User(name='mary', fullname='Mary Contrary', password='xxg527'),
    User(name='fred', fullname='Fred Flinstone', password='blah')
])

In [25]:
ed_user.password = 'f8s7ccs'

print(session.new)

IdentitySet([<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>, <User(name='wendy', fullname='Wendy Williams', password='foobar')>, <User(name='mary', fullname='Mary Contrary', password='xxg527')>, <User(name='fred', fullname='Fred Flinstone', password='blah')>])


In [26]:
session.commit()

2017-04-21 12:01:48,485 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-04-21 12:01:48,488 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-21 12:01:48,491 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'f8s7ccs')
2017-04-21 12:01:48,494 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-21 12:01:48,495 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'foobar')
2017-04-21 12:01:48,497 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-21 12:01:48,498 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'xxg527')
2017-04-21 12:01:48,500 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-21 12:01:48,501 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flinstone', 'blah')
2017-04-21 12:01:48,503 INFO sqlalchemy.engine.base.Engine COMMIT

In [27]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)
    print(instance)

2017-04-21 12:01:48,550 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-04-21 12:01:48,558 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
2017-04-21 12:01:48,573 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>
wendy Wendy Williams
<User(name='wendy', fullname='Wendy Williams', password='foobar')>
mary Mary Contrary
<User(name='mary', fullname='Mary Contrary', password='xxg527')>
fred Fred Flinstone
<User(name='fred', fullname='Fred Flinstone', password='blah')>


In [28]:
print(session.query(User).filter(User.name == 'ed').all())

2017-04-21 12:01:48,587 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
2017-04-21 12:01:48,589 INFO sqlalchemy.engine.base.Engine ('ed',)
[<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>]


In [29]:
print(session.query(User).filter(User.name != 'ed').all())

2017-04-21 12:01:48,617 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name != ?
2017-04-21 12:01:48,620 INFO sqlalchemy.engine.base.Engine ('ed',)
[<User(name='wendy', fullname='Wendy Williams', password='foobar')>, <User(name='mary', fullname='Mary Contrary', password='xxg527')>, <User(name='fred', fullname='Fred Flinstone', password='blah')>]


In [30]:
print(session.query(User).filter(User.name.like('%ed%')).all())

2017-04-21 12:01:48,648 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ?
2017-04-21 12:01:48,651 INFO sqlalchemy.engine.base.Engine ('%ed%',)
[<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>, <User(name='fred', fullname='Fred Flinstone', password='blah')>]


In [31]:
session.query(User).filter(User.name.like('%ed')).count()

2017-04-21 12:01:48,676 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ?) AS anon_1
2017-04-21 12:01:48,678 INFO sqlalchemy.engine.base.Engine ('%ed',)


2

In [32]:
session.close()

2017-04-21 12:01:48,695 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [33]:
%%bash

cat alchemy.db

   o o                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

## Дополнительные материалы:

  - Python Database API Specification https://www.python.org/dev/peps/pep-0249/
  - Peewee http://peewee.readthedocs.io/en/latest/index.html
  - Django ORM: http://www.effectivedjango.com/orm.html